<h1>Open Street Map SF Study</h1>

<h2> Map </h2>
<p> Links to the map area I am discussing as well as a description of that area are in the text file 'osm-sfmap.txt'.
The map covers the region around the bay as well as the city of San Francisco, CA. Therefore, we will see cities in the data as far as Palo Alto and Berkeley.</p> 

<h2> Code Functionality</h2>
<p>I have three functions in audit.py that I use to audit the open street map data. In this project I only focus on the key-value pairs for the nodes tags. Each function deals with one value at a time. I dealt with the keys that I thought were the most problematic which were:</p>
<br>
<li>Phone Numbers</li>
<li>Street Address</li>
<li>Source</li>

<p> I also have other functions I use to query the database for statistics </p>
<p>All database querying was done using sqlite3</p>

In [7]:
import sqlite3
db = sqlite3.connect("SanFranciscoOSM.db")
cur = db.cursor()

<h2>Problematic Areas</h2>
<h3><u>Phone Numbers</u></h3>
<br>
I came across a variety of formats for phone numbers. The first thing I wanted to do was put them all in the same format. Some phone numbers had letters so I included all alpha-numeric characters and placed them in the form of '+1-###-###-####'. I noticed that some numbers were missing digits or contained something other than a phone number (like a website). These were returned as None.

Some formats that I saw were:
<li>123-456-7890 (missing the country code)</li>
<li>1234567890 (missing dashes)</li>
<li>123-456-789 (missing digits)</li>

Except for the last number, all of the above numbers would be formatted as '+1-123-456-7890'. The last number would return None.

In [1]:
def audit_phones(phone):
    """ Returns all phones that are able to be formatted in a similar way"""
    
    alphanumeric = re.compile('[a-zA-Z0-9]*')
    good_phone_format = re.compile('\+1\-[0-9A-Za-z]{3}\-[0-9A-Za-z]{3}\-[0-9A-Za-z]{4}')

    phone = re.findall(alphanumeric, phone)  # Gets alphanumeric characters in string
    phone = ''.join(phone)
    
    
    if phone[0] != '1': # +1 is in U.S. numbers, add this number if it's not already present
        phone = '1'+phone
        
    if len(phone) == 11:  # Some phone numbers are missing digits, so check if there are 11
        phone = '+'+phone[0]+'-'+phone[1:4]+'-'+phone[4:7]+'-'+phone[7:]
        
    if not re.match(good_phone_format, phone):  # If the phone does not match the proper format, return None
        return None
    else:
        return phone

<h3><u>Street Addresses</u></h3>
<br>
When auditing street addresses, I decided to focus on abbreviations, variations in capitilization, and punctuation. I did not translate cardinal directions, because there were streets that actually contained the single letter 'E' but did not translate to East. I knew this because there were other streets that started with single letters (e.g. - 'D Street').

Here are some examples of how they will be converted:
<li>E Street -- (E Street)</li>
<li>Upton St. -- ('Upton Street')</li>
<li>foothill blvd -- ('Foothill Boulevard')</li>
<li>Woodside Road, Suite 100 -- ('Woodside Road')</li>
<li>A st. & N blvd -- (('A Street', 'N Boulevard'))</li>

I removed numbers in the beginning and ending of street names, as well as the word 'suite' or 'ste'. I realized that most people did not put in their exact addresses. I reasoned that having generalized street names would make them easier to analyze as groups.
Also, if there were two addresses present, I would return them as a tuple.

In [2]:
good_street_names = {'rd': 'Road',
                     'plz': 'Plaza',
                     'blvd': 'Boulevard',
                     'ave': 'Avenue',
                     'st': 'Street',
                     'hwy': 'Highway',
                     'ctr': 'Center',
                     'dr': 'Drive'}

In [3]:
def audit_street(street):
    """Makes sure that the streets only contain the street name in its full,
     un-abbreviated form"""
    
    numbers_only = re.compile('^[0-9]*$')  # this will match to strings that only have numbers
    street = street.replace('.', '').replace('#', '').replace(',', '')  # remove common punctuation

    st_words = street.split(' ')
    first, last = st_words[0], st_words[-1]

    for j, word in enumerate(st_words):
        # Capitalize the first letter in each word
        st_words[j] = word.strip().title()

        # return a tuple of the street names if given two streets
        if word.lower() == 'and' or word.lower() == '&':
            double = street.split(word)
            return audit_street(double[0]), audit_street(double[1])

        # remove all numbers at the beginning and ending of string
        if word == first or word == last:
            if re.match(numbers_only, word):
                st_words[j] = ''

        # Remove suite from the street names.
        if word == 'Suite' or word == 'Ste':
            st_words[j] = ''

        # Replace street abbreviations with full name
        if word.lower().strip() in good_street_names.keys():
            st_words[j] = good_street_names[word.lower()]
            
    if '' in st_words:
        st_words.remove('')
        
    return ' '.join(st_words).strip()

<h3><u>Source</u></h3>
<br>
The sources for each node refer to the source of information for a specific node. Many of the sources were very specific. In order to be able to group each source into a category, I glanced through the unique sources and came up with a list of the most common sources by their most simplistic names. I also did the same with websites, by making it less specific.

All sources that were not converted stay exactly as they are.

Here are some examples of how they will be converted:
<li>Field Observation -- ('observation')</li>
<li>Google; Yahoo -- (('google', 'yahoo')) </li>
<li>NPS map -- ('map')</li>
<li>http://www.dot.ca.gov/ ... /101.pdf -- ('http://www.dot/ca/gov/')</li>
<li>Street View -- ('Street View')</li> 

<i>Note</i>: The '...' in the 4th bullet was used for convenience and is not actually what is written


In [4]:
common_sources = ['bing', 'yahoo', 'google', 'survey', 'map', 'observation',
                  'knowledge', 'gps', 'yelp', 'website', 'gtfs', 'usgs']

common_webs = ['dot.ca.gov', 'yelp.com/biz']

In [5]:
def audit_sources(source):
    """This cleans the sources of information, by attempting to reduce variation"""
    
    if (';' in source) or (type(source) == tuple):
        # If there is more than one source, it's usually separated by a ';'. Return both
        try:
            double = source.split(';')
        except:
            double = source
        return audit_sources(double[0]), audit_sources(double[1])
    
    # If it's not a website, clean the string
    if 'http' not in source:
        source = source.replace('_', '')
        # If there is a common source in the string, replace it with the common source
        for cs in common_sources:
            if cs in source.lower():
                source = source.replace(source, cs)
    else:
        # If this a common website, make it less specific so you can group them together
        for cs in common_webs:
            if cs in source:
                source = source.rsplit('/', 1)[0]
    return source.lower().strip()

<h2>Statistics</h2>

<h3><u>File Size</u></h3>

<i>San Francisco OSM -- 400.722 MB</i>

<h3><u>Number of Nodes</u></h3>

In [9]:
cur.execute("SELECT COUNT(*) FROM nodes")
for c in cur: print c[0]

4533476


<h3><u>Number of Ways</u></h3>

In [10]:
cur.execute("SELECT COUNT(*) FROM ways")
for c in cur: print c[0]

524685


<h3><u>Top 10 keys for Nodes</u></h3>

In [15]:
cur.execute("""
            SELECT k, COUNT(*) FROM nodes_tags 
            GROUP BY k 
            ORDER BY COUNT(*) DESC 
            LIMIT 10
                    """)
for c in cur: print c[1], '|', c[0]

addr:housenumber
highway
addr:street
addr:city
name
created_by
amenity
source
addr:postcode
addr:state


<h3><u>Top 10 keys for Ways</u></h3>

In [16]:
cur.execute("""
            SELECT k, COUNT(*) FROM ways_tags 
            GROUP BY k 
            ORDER BY COUNT(*) DESC 
            LIMIT 10
                    """)

for c in cur: print c[1], '|', c[0]

building
highway
name
source
tiger:county
tiger:cfcc
tiger:name_base
tiger:name_type
height
tiger:zip_left


<h3><u>Top Cities in the file</u></h3>

In [17]:
# This gets the count for every node and way that contains a key for a city.
cur.execute("""
            SELECT COUNT(*) FROM 
            (SELECT v from nodes_tags WHERE k='addr:city' 
            UNION ALL
            SELECT v from ways_tags WHERE k='addr:city')
                                                        """)

v_count = [c[0] for c in cur]


# This gets the top ten cities for both nodes and ways and their count
cur.execute("""
            SELECT v, COUNT(*) as count FROM
            (SELECT v from nodes_tags WHERE k='addr:city' 
            UNION ALL
            SELECT v from ways_tags WHERE k='addr:city')
            GROUP BY v
            ORDER BY count DESC
            LIMIT 10
                                                        """)

# Here, I get a percentage by dividing the city's count with v_count and multiplying by 100. 
# The result is formatted to 2 decimal places
for c in cur:
        print c[0], "|", "{0:.2f}".format((c[1]*100.0)/v_count[0])+'%'

Redwood City | 41.44%
San Francisco | 30.28%
Berkeley | 9.90%
Piedmont | 6.71%
Palo Alto | 2.89%
Richmond | 2.38%
Oakland | 2.36%
Union City | 0.46%
Albany | 0.39%
Burlingame | 0.35%
